# LangChain Expression Language

In [ ]:
%pip install langchain langchain_openai langchain-community --upgrade

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

Explain the basic syntax of [LangChain Expression Language](https://python.langchain.com/docs/expression_language/), which uses the pipe symbol `|` to connect components. Each component represents a specific task or action.

To make it as easy as possible to create custom chains, LangChain implemented a `Runnable` protocol. The Runnable protocol is implemented for most components. This is a standard interface, which makes it easy to define custom chains as well as invoke them in a standard way. The standard interface includes:

- `stream`: stream back chunks of the response
- `invoke`: call the chain on an input
- `batch`: call the chain on a list of inputs

These also have corresponding async methods:

- `astream`: stream back chunks of the response async
- `ainvoke`: call the chain on an input async
- `abatch`: call the chain on a list of inputs async
- `astream_log`: stream back intermediate steps as they happen, in addition to the final response

----

## The Runnable Protocol:

A unit of work that can be invoked, batched, streamed, transformed and composed.

All methods accept an optional config argument, which can be used to configure execution, add tags and metadata for tracing and debugging etc.

Runnables expose schematic information about their input, output and config via the input_schema property, the output_schema property and config_schema method.

The LangChain Expression Language (LCEL) is a declarative way to compose Runnables into chains. Any chain constructed this way will automatically have sync, async, batch, and streaming support.

The main composition primitives are `RunnableSequence` and `RunnableParallel`.

---

## RunnableLambda

In [63]:
from langchain_core.runnables import RunnableLambda

print(type(RunnableLambda(lambda x: x + 1))) # <class 'langchain.schema.runnable.RunnableLambda'>

<class 'langchain.schema.runnable.base.RunnableLambda'>


In [65]:
chain = RunnableLambda(lambda x: x + 1) 

In [67]:
chain.invoke(1)

2

In [68]:
chain.invoke(5)

6

## RunnableSequence

`RunnableSequence` invokes a series of runnables sequentially, with one runnable’s output serving as the next’s input. Construct using the `|` operator or by passing a list of runnables to RunnableSequence.

In [69]:
# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | (lambda x: x * 2)

print(type(sequence)) # <class 'langchain.schema.runnable.RunnableSequence'>
print('\n\n---')
print(sequence.invoke(1)) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]

<class 'langchain.schema.runnable.base.RunnableSequence'>


---
4


[4, 6, 8]

## RunnableParallel

The `RunnableParallel`, allows for multiple runnables to be invoked in parallel, construct using a dictionary of runnables to invoke in parallel.

In [70]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    "mul_2": RunnableLambda(lambda x: x * 2),
    "mul_5": RunnableLambda(lambda x: x * 5),
}
sequence.invoke(1)  # {'mul_2': 4, 'mul_5': 10}

{'mul_2': 4, 'mul_5': 10}

---------------

## Combining the output of multiple runnables into a single response

A sequence that contains a RunnableParallel constructed using a dict literal, this is then followed by a RunnableLambda that consumes the output of the RunnableParallel

In [71]:
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
} | RunnableLambda(lambda x: x['mul_2'] + x['mul_5'])
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

14

In [72]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel({
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
})

# This is a dictionary, however it will be composed with other runnables when used in a sequence:
parallel_two = {
    'mul_2': RunnableLambda(lambda x: x['input_one'] * 2),
    'mul_5': RunnableLambda(lambda x: x['input_two'] * 5)
}

print(type(parallel)) # <class 'langchain.schema.runnable.RunnableParallel'>
print(type(parallel_two)) # <class 'dict'>

<class 'langchain.schema.runnable.base.RunnableParallel'>
<class 'dict'>


In [73]:
chain = parallel | RunnableLambda(lambda x: x['mul_2'] + x['mul_5']) 
chain.invoke(5)

35

In [54]:
second_chain = parallel_two | RunnableLambda(lambda x: x['mul_2'] + x['mul_5']) 
second_chain.invoke({'input_one': 5, 'input_two': 10})

60

------------------------------------------------

### You only need a _`Runnable` at the start_, you can use other Python functions _after the first `Runnable`_

Technically you only need a `RunnableLambda` or `RunnableParallel` as the first expression after that you can use Python functions:

In [74]:
parallel = RunnableParallel({
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
})

# This is bad practice:
test = lambda x : x + 1  | parallel
print(test)
test.invoke(5)

<function <lambda> at 0x7f9a1019a160>


AttributeError: 'function' object has no attribute 'invoke'

In [75]:
# This is good practice:
test = RunnableLambda(lambda x: x + 1) | parallel
print(test)
test.invoke(5)

first=RunnableLambda(lambda x: x + 1) last={
  mul_2: RunnableLambda(...),
  mul_5: RunnableLambda(...)
}


{'mul_2': 12, 'mul_5': 30}

---

## Combining Steps in A Runnable

In [88]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel({
    'item_one': RunnableLambda(lambda x: f"Hello {x['name']} "),
    'item_two': RunnableLambda(lambda x: 'Welcome to the World!')
})


In [89]:
def combine(x):
    return x['item_one'] + x['item_two']

In [90]:
parallel_chain_example = parallel | combine
parallel_chain_example.invoke({'name': "James"}) 

'Hello James Welcome to the World!'

In [83]:
lambda_example = RunnableLambda(lambda x: {'item_one': 'Hello ', 'item_two': 'World'})
lambda_chain_example = lambda_example | combine
lambda_chain_example.invoke({})

'Hello World'